## Artificial and Computational Intelligence Assignment 2

## Gaming with Min-Max Algorithm - Solution template

### Submitted by Group 1 with 100% contribution from all participants:

1. Name: Reddy Balaji .C (BITS ID: 2023AC05862); Email: 2023ac05862@wilp.bits-pilani.ac.in
2. Name: Saurabh Jalendra (BITS ID: 2023AC05912); Email: 2023ac05912@wilp.bits-pilani.ac.in
3. Name: Tushar Shandilya (BITS ID: 2023AC05573); Email: 2023ac05573@wilp.bits-pilani.ac.in
4. Name:  Bandana Kumari (BITS ID: 2023AC05879); Email: 2023ac05879@wilp.bits-pilani.ac.in
5. Name: Monica Malik (BITS ID: 2023AC05875); Email: 2023ac05875@wilp.bits-pilani.ac.in

# Things to follow

1. Use appropriate data structures to represent the graph using python libraries
2. Provide proper documentation
3. Create neat solution without error during game playing

### Coding begins here

### PEAS - Data structures and fringes that define the Agent environment goes here

In [37]:
class CatchUpGame:
    def __init__(self, n):
        self.n = n
        self.available_numbers = list(range(1, n + 1))
        self.P1_sum = 0
        self.P2_sum = 0
        self.turn = "P1"

    # Check if all numbers have been selected
    def game_over(self):
        return len(self.available_numbers) == 0

    # Display the current board and scores
    def display_game_state(self):
        print(f"Available Numbers: {self.available_numbers}")
        print(f"P1 Sum: {self.P1_sum} | P2 Sum: {self.P2_sum}")

    # Perform a move for a player
    def make_move(self, player, choices):
        for choice in choices:
            if choice in self.available_numbers:
                self.available_numbers.remove(choice)
                if player == "P1":
                    self.P1_sum += choice
                else:
                    self.P2_sum += choice

    # Calculate if a player can continue to pick numbers
    def can_continue_choosing(self, player):
        if player == "P1":
            return self.P1_sum <= self.P2_sum
        else:
            return self.P2_sum <= self.P1_sum


### Implementation of the Min-Max algorithm

In [38]:
# Min-Max algorithm modified for multiple picks in one turn
def minimax(game, depth, is_maximizing, sum_current_player, sum_opponent):
    # Base case: game over
    if game.game_over():
        return game.P1_sum - game.P2_sum  # Evaluation of the game state

    best_score = float('-inf') if is_maximizing else float('inf')
    best_choices = []

    for num in game.available_numbers:
        new_game = CatchUpGame(game.n)
        new_game.available_numbers = game.available_numbers.copy()
        new_game.P1_sum = game.P1_sum
        new_game.P2_sum = game.P2_sum

        player = "P1" if is_maximizing else "P2"
        current_sum = sum_current_player
        choices = []

        while new_game.can_continue_choosing(player):
            new_game.make_move(player, [num])
            choices.append(num)
            current_sum += num

            # Recursively calculate the score for the next state
            score = minimax(new_game, depth + 1, not is_maximizing, sum_opponent, current_sum)
            if is_maximizing:
                if score > best_score:
                    best_score = score
                    best_choices = choices.copy()
            else:
                if score < best_score:
                    best_score = score
                    best_choices = choices.copy()

            if current_sum > sum_opponent:
                break

    return best_score


### Implementation of the alpha-beta pruning  

In [39]:
# Alpha-Beta Pruning algorithm
def alpha_beta(game, depth, alpha, beta, is_maximizing, sum_current_player, sum_opponent):
    if game.game_over():
        return game.P1_sum - game.P2_sum  # Evaluation of the game state

    best_score = float('-inf') if is_maximizing else float('inf')
    
    for num in game.available_numbers:
        new_game = CatchUpGame(game.n)
        new_game.available_numbers = game.available_numbers.copy()
        new_game.P1_sum = game.P1_sum
        new_game.P2_sum = game.P2_sum

        player = "P1" if is_maximizing else "P2"
        current_sum = sum_current_player
        choices = []

        while new_game.can_continue_choosing(player):
            new_game.make_move(player, [num])
            choices.append(num)
            current_sum += num

            # Recursively calculate the score for the next state using alpha-beta pruning
            score = alpha_beta(new_game, depth + 1, alpha, beta, not is_maximizing, sum_opponent, current_sum)

            if is_maximizing:
                best_score = max(best_score, score)
                alpha = max(alpha, best_score)
            else:
                best_score = min(best_score, score)
                beta = min(beta, best_score)

            if beta <= alpha:
                break

            if current_sum > sum_opponent:
                break

    return best_score


### Choice and implementation of the Static Evaluation Function.

In [40]:
# Static evaluation function simply returns the difference between P1 and P2 sums
def evaluate_game_state(game):
    return game.P1_sum - game.P2_sum


In [41]:
# Get the best move for the computer (P1) using Min-Max or Alpha-Beta
def get_best_move(game, is_maximizing, use_alpha_beta=False):
    best_score = float('-inf') if is_maximizing else float('inf')
    best_choices = []

    sum_current_player = game.P1_sum if is_maximizing else game.P2_sum
    sum_opponent = game.P2_sum if is_maximizing else game.P1_sum

    for num in game.available_numbers:
        new_game = CatchUpGame(game.n)
        new_game.available_numbers = game.available_numbers.copy()
        new_game.P1_sum = game.P1_sum
        new_game.P2_sum = game.P2_sum

        player = "P1" if is_maximizing else "P2"
        choices = []
        current_sum = sum_current_player

        while new_game.can_continue_choosing(player):
            new_game.make_move(player, [num])
            choices.append(num)
            current_sum += num

            # Evaluate the move using Min-Max or Alpha-Beta pruning
            if use_alpha_beta:
                score = alpha_beta(new_game, 0, float('-inf'), float('inf'), not is_maximizing, sum_opponent, current_sum)
            else:
                score = minimax(new_game, 0, not is_maximizing, sum_opponent, current_sum)

            if is_maximizing:
                if score > best_score:
                    best_score = score
                    best_choices = choices.copy()
            else:
                if score < best_score:
                    best_score = score
                    best_choices = choices.copy()

            if current_sum > sum_opponent:
                break

    return best_choices

# Interactive game loop
def play_game(game, use_alpha_beta=False):
    while not game.game_over():
        game.display_game_state()

        if game.turn == "P1":
            # P1 can pick numbers until P1_sum >= P2_sum
            best_moves = []
            while game.P1_sum <= game.P2_sum and not game.game_over():
                # Get the best move for P1 (computer) using Min-Max or Alpha-Beta
                best_move = get_best_move(game, True, use_alpha_beta)
                best_moves.extend(best_move)
                print(f"P1 (Computer) chooses {best_move}")
                game.make_move("P1", best_move)

            game.turn = "P2"
        else:
            # Get user input for P2 (human player) until P2_sum >= P1_sum
            print("It's your turn, P2!")
            sum_p1 = game.P1_sum
            user_choices = []
            while game.P2_sum <= sum_p1 and not game.game_over():
                try:
                    choice = int(input(f"Choose a number from {game.available_numbers}: "))
                    if choice in game.available_numbers:
                        user_choices.append(choice)
                        game.make_move("P2", [choice])
                    else:
                        print("Invalid choice. Please choose a valid number.")
                except ValueError:
                    print("Please enter a valid number.")
            game.turn = "P1"

    # Game over, declare the winner
    game.display_game_state()
    if game.P1_sum > game.P2_sum:
        print("P1 (Computer) wins!")
    elif game.P2_sum > game.P1_sum:
        print("P2 (You) win!")
    else:
        print("It's a tie!")


In [42]:
# Start the game with user input for n and option to use Alpha-Beta pruning
n = int(input("Enter the highest number (n) to play with: "))
use_alpha_beta = input("Do you want to use Alpha-Beta pruning? (yes/no): ").lower() == 'yes'
game = CatchUpGame(n)
play_game(game, use_alpha_beta)


Enter the highest number (n) to play with:  10
Do you want to use Alpha-Beta pruning? (yes/no):  no


Available Numbers: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
P1 Sum: 0 | P2 Sum: 0
P1 (Computer) chooses [1]
Available Numbers: [2, 3, 4, 5, 6, 7, 8, 9, 10]
P1 Sum: 1 | P2 Sum: 0
It's your turn, P2!


Choose a number from [2, 3, 4, 5, 6, 7, 8, 9, 10]:  6


Available Numbers: [2, 3, 4, 5, 7, 8, 9, 10]
P1 Sum: 1 | P2 Sum: 6
P1 (Computer) chooses [2]
P1 (Computer) chooses [3]
P1 (Computer) chooses [4]
Available Numbers: [5, 7, 8, 9, 10]
P1 Sum: 10 | P2 Sum: 6
It's your turn, P2!


Choose a number from [5, 7, 8, 9, 10]:  5


Available Numbers: [7, 8, 9, 10]
P1 Sum: 10 | P2 Sum: 11
P1 (Computer) chooses [8]
Available Numbers: [7, 9, 10]
P1 Sum: 18 | P2 Sum: 11
It's your turn, P2!


Choose a number from [7, 9, 10]:  7
Choose a number from [9, 10]:  10


Available Numbers: [9]
P1 Sum: 18 | P2 Sum: 28
P1 (Computer) chooses [9]
Available Numbers: []
P1 Sum: 27 | P2 Sum: 28
P2 (You) win!
